# Title: Binance data feed
## Purpose: 
To download data from Binance Cryptocurrency Derivatives Exchange platform using API
## Requirements: 
API account on Binance, Python, Excel
## Steps: 
  S1. Connect to Binance \
  S2. Download the data \
  S3. Clean the data \
  S4. Transform the data \
  S5. Export the data to excel

In [ ]:
import pandas as pd
import xlwings as xw
import binance
from binance.client import Client

# Keys
api_key = 'z'
api_secret = 'PwOL04okyC8paI561da1z9XhsxqJjm1yRx629ov9ydDu22cVknV7BCHwHcfnBbRB'

# Connect to Binance
client = Client(api_key, api_secret)

#binance.fetch_balance({'recvWindow': 10000000})

## STEP 1
# Get account status
info = client.get_account()
df = pd.DataFrame(info['balances'])

# Trasnformation
# Change type
df["free"] = df.free.astype(float)
# Filter by items with values
df1 = df.loc[df['free'] > 0]
# Rename columns
df1 = df1.rename(columns = {'asset': 'symbol', 'free': 'value'}, inplace = False)
# Select columns
df1 = df1[['symbol','value']]

## STEP 2
# Get current price
df2 = pd.DataFrame(client.get_all_tickers())
# Remove USDT from ticker name
df2["symbol"]= df2["symbol"].str.split("USDT", n = 1, expand = True)
# Change type
df2["price"] = df2.price.astype(float)

## STEP 3
# Merge two datasets
df3 = pd.merge(df1, df2, how = 'left')
# fill NaN with 1 --> this is for USDT as can not generate by crypto_list
df3['price'].fillna(1, inplace=True)

# Get the USD value
df3['value'] = df3['price']*df3['value']
df3['source'] = 'Binance'
df3['currency'] = 'USD/PLN'

df_Bianance_final = df3[['source','symbol','value','currency']]

#wb = xw.Book('E:\Gielda\Porfel\Porfel.xlsm')
#sheet = wb.sheets['pyScript']
#sheet.range('A22').value = df_Bianance_final